In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow 
#importing models, layers and metrics (not used for the moment)
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input, concatenate
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
#data processing 

#train set
train_features = pd.read_csv("./data/train.csv")

features = []
for i in range(len(train_features)):
    features.append(list(train_features["Sequence"][i]))

features_pd = pd.DataFrame(features)
final = pd.concat([features_pd, train_features['Active']], axis=1)


#test set 
test_features = pd.read_csv("./data/test.csv")

tfeatures = []
for i in range(len(test_features)):
    tfeatures.append(list(test_features["Sequence"][i]))

tfeatures_pd = pd.DataFrame(tfeatures)
final_test = tfeatures_pd

In [3]:
#list of amino acids 


categ=['A', 'C', 'D', 'F', 'G', 'R', 'H', 'K', 'E','S', 'T', 'N', 'Q', 'U', 'P', 'I', 'L', 'M', 'W', 'Y', 'V']
categ.sort()
print(categ)

categories=[categ, categ,categ, categ]

['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y']


In [4]:
#getting the features 
features=final.iloc[:, :4].astype(str)
features_test=final_test.astype(str)

enc = OneHotEncoder(sparse=False, categories=categories)
transformed_features=enc.fit_transform(features)
transformed_test=enc.fit_transform(features_test)

In [5]:
transformed_features.shape

(112000, 84)

In [6]:
#reshaping input 
train_samples = transformed_features.reshape(transformed_features.shape[0],1, transformed_features.shape[1])
test_samples =transformed_test.reshape(transformed_test.shape[0],1, transformed_test.shape[1])
#reshaping labels
labels=np.array(final['Active'])
print(labels)

[0 0 0 ... 0 0 0]


In [7]:
#define parameters
samples=112000
features=84
fnode_1=69
fnode_2=42
fnode_3=13

#cin = Input(batch_shape=(None,2))
pin = Input(batch_shape=(None, 1, features))

#cout = cin
#pout = pin


layer_1 = Dense(fnode_1, kernel_initializer='he_uniform', activation='relu')(pin)
#out = pout

#out = concatenate([pout, cout])
layer_2 = Dense(fnode_2, activation='relu')(layer_1)
layer_3 = Dense(fnode_3, activation='relu')(layer_2)


out = Dropout(0.8)(layer_3)
out = Dense(1, kernel_initializer='he_uniform', activation='sigmoid')(out)

adam=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01)
    
model = Model(inputs=[pin], outputs=[out])

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    
#summarise the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 84)]           0         
_________________________________________________________________
dense (Dense)                (None, 1, 69)             5865      
_________________________________________________________________
dense_1 (Dense)              (None, 1, 42)             2940      
_________________________________________________________________
dense_2 (Dense)              (None, 1, 13)             559       
_________________________________________________________________
dropout (Dropout)            (None, 1, 13)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1, 1)              14        
Total params: 9,378
Trainable params: 9,378
Non-trainable params: 0
___________________________________________________________

In [8]:
callback = EarlyStopping(monitor='loss')
model_fit = model.fit(train_samples, labels, epochs=50, callbacks = [callback])

Epoch 1/50
3500/3500 [==============================] - 4s 995us/step - loss: 0.2971 - accuracy: 0.9509
Epoch 2/50
3500/3500 [==============================] - 3s 888us/step - loss: 0.2266 - accuracy: 0.9622
Epoch 3/50
3500/3500 [==============================] - 3s 887us/step - loss: 0.2152 - accuracy: 0.9624
Epoch 4/50
3500/3500 [==============================] - 3s 896us/step - loss: 0.2117 - accuracy: 0.9614
Epoch 5/50
3500/3500 [==============================] - 3s 895us/step - loss: 0.2031 - accuracy: 0.9625
Epoch 6/50
3500/3500 [==============================] - 3s 920us/step - loss: 0.2001 - accuracy: 0.9621
Epoch 7/50
3500/3500 [==============================] - 3s 900us/step - loss: 0.1985 - accuracy: 0.9619
Epoch 8/50
3500/3500 [==============================] - 3s 911us/step - loss: 0.1961 - accuracy: 0.9625
Epoch 9/50
3500/3500 [==============================] - 4s 1ms/step - loss: 0.1937 - accuracy: 0.9636
Epoch 10/50
3500/3500 [==============================] - 4s 1ms/st

In [9]:
# predict 
pred = model.predict(test_samples)

In [10]:
print(np.sum(labels))

4213


In [11]:
np.sum(np.argmax(pred,axis=1))

0

In [17]:

model2 = Sequential()
model2.add(Dense(12, input_dim=84, activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
# compile the keras model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model2.fit(transformed_features, labels, epochs=20)


Epoch 1/20
3500/3500 [==============================] - 4s 913us/step - loss: 0.1392 - accuracy: 0.9626
Epoch 2/20
3500/3500 [==============================] - 3s 787us/step - loss: 0.0483 - accuracy: 0.9830
Epoch 3/20
3500/3500 [==============================] - 3s 810us/step - loss: 0.0381 - accuracy: 0.9865
Epoch 4/20
3500/3500 [==============================] - 3s 809us/step - loss: 0.0326 - accuracy: 0.9873
Epoch 5/20
3500/3500 [==============================] - 3s 917us/step - loss: 0.0298 - accuracy: 0.98820s - loss:
Epoch 6/20
3500/3500 [==============================] - 3s 904us/step - loss: 0.0270 - accuracy: 0.9900
Epoch 7/20
3500/3500 [==============================] - 3s 796us/step - loss: 0.0259 - accuracy: 0.9898
Epoch 8/20
3500/3500 [==============================] - 3s 923us/step - loss: 0.0240 - accuracy: 0.9905
Epoch 9/20
3500/3500 [==============================] - 3s 779us/step - loss: 0.0240 - accuracy: 0.9905
Epoch 10/20
3500/3500 [==============================]

In [18]:
pred2=model2.predict_classes(transformed_test)

/Users/darialaslo/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [23]:
pd.DataFrame(pred2)

,0
0,0
1,0
2,0
3,0
4,0
...,...
47995,0
47996,0
47997,0
47998,0


In [21]:
transformed_test.shape

(48000, 84)